# ULMFit

In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [ ]:
#export
from exp.nb_12a import *

## Data

In [ ]:
path = datasets.untar_data(datasets.URLs.IMDB)

In [ ]:
ll = pickle.load(open(path/'ll_lm.pkl', 'rb'))

In [ ]:
bs,bptt = 128,70
data = lm_databunchify(ll, bs, bptt)

In [ ]:
vocab = ll.train.proc_x[1].vocab

## Finetuning the LM

Before tackling the classification task, we have to finetune our language model to the IMDB corpus. Make sure you have the pretrained.pth and vocab.pkl files in your IMDB data folder. 

In [ ]:
path.ls()

[PosixPath('/home/ubuntu/.fastai/data/imdb/imdb.vocab'),
 PosixPath('/home/ubuntu/.fastai/data/imdb/vocab_lm.pkl'),
 PosixPath('/home/ubuntu/.fastai/data/imdb/finetuned_enc.pth'),
 PosixPath('/home/ubuntu/.fastai/data/imdb/data_lm.pkl'),
 PosixPath('/home/ubuntu/.fastai/data/imdb/tmp_lm'),
 PosixPath('/home/ubuntu/.fastai/data/imdb/models'),
 PosixPath('/home/ubuntu/.fastai/data/imdb/vocab.pkl'),
 PosixPath('/home/ubuntu/.fastai/data/imdb/ll_clas.pkl'),
 PosixPath('/home/ubuntu/.fastai/data/imdb/test'),
 PosixPath('/home/ubuntu/.fastai/data/imdb/pretrained.pth'),
 PosixPath('/home/ubuntu/.fastai/data/imdb/unsup'),
 PosixPath('/home/ubuntu/.fastai/data/imdb/tmp_clas'),
 PosixPath('/home/ubuntu/.fastai/data/imdb/finetuned.pth'),
 PosixPath('/home/ubuntu/.fastai/data/imdb/README'),
 PosixPath('/home/ubuntu/.fastai/data/imdb/ll_lm.pkl'),
 PosixPath('/home/ubuntu/.fastai/data/imdb/train')]

In [ ]:
dps = tensor([0.25, 0.1, 0.2, 0.02, 0.15]) * 0.5

In [ ]:
emb_sz, nh, nl = 300, 300, 2
model = get_language_model(len(vocab), emb_sz, nh, nl, 0, input_p=dps[0], output_p=dps[1], weight_p=dps[2], 
                           embed_p=dps[3], hidden_p=dps[4])

Match embeddings

In [ ]:
old_wgts  = torch.load(path/'pretrained.pth')
old_vocab = pickle.load(open(path/'vocab.pkl', 'rb'))

In [ ]:
vocab.index('house'),old_vocab.index('house')

(347, 230)

In [ ]:
house_wgt  = old_wgts['0.encoder.weight'][230]
house_bias = old_wgts['1.decoder.bias'][230] 

In [ ]:
def match_embeds(old_wgts, old_vocab, new_vocab):
    wgts = old_wgts['0.encoder.weight']
    bias = old_wgts['1.decoder.bias']
    wgts_m,bias_m = wgts.mean(dim=0),bias.mean()
    new_wgts = wgts.new_zeros(len(new_vocab), wgts.size(1))
    new_bias = bias.new_zeros(len(new_vocab))
    for i,w in enumerate(new_vocab): 
        if w in old_vocab:
            idx = old_vocab.index(w)
            new_wgts[i],new_bias[i] = wgts[idx],bias[idx]
        else: new_wgts[i],new_bias[i] = wgts_m,bias_m
    old_wgts['0.encoder.weight']    = new_wgts
    old_wgts['0.encoder_dp.emb.weight'] = new_wgts
    old_wgts['1.decoder.weight']    = new_wgts
    old_wgts['1.decoder.bias']      = new_bias
    return old_wgts

In [ ]:
wgts = match_embeds(old_wgts, old_vocab, vocab)

In [ ]:
assert torch.allclose(wgts['0.encoder.weight'][347],house_wgt)
assert torch.allclose(wgts['1.decoder.bias'][347],house_bias)

In [ ]:
model.load_state_dict(wgts)

In [ ]:
torch.save(model.state_dict, path/'tmp_clas'/'init.pth')

Split

In [ ]:
model

SequentialRNN(
  (0): AWD_LSTM(
    (encoder): Embedding(60003, 300, padding_idx=0)
    (encoder_dp): EmbeddingDropout(
      (emb): Embedding(60003, 300, padding_idx=0)
    )
    (rnns): ModuleList(
      (0): WeightDropout(
        (module): LSTM(300, 300, batch_first=True)
      )
      (1): WeightDropout(
        (module): LSTM(300, 300, batch_first=True)
      )
    )
    (input_dp): RNNDropout()
    (hidden_dps): ModuleList(
      (0): RNNDropout()
      (1): RNNDropout()
    )
  )
  (1): LinearDecoder(
    (decoder): Linear(in_features=300, out_features=60003, bias=True)
    (output_dp): RNNDropout()
  )
)

In [ ]:
def lm_splitter(m):
    groups = []
    for i in range(len(m[0].rnns)): groups.append(nn.Sequential(m[0].rnns[i], m[0].hidden_dps[i]))
    groups = [nn.Sequential(m[0].encoder, m[0], m[0].input_dp, m[1])]
    return [list(o.parameters()) for o in groups]

In [ ]:
for rnn in model[0].rnns:
    for p in rnn.parameters(): p.requires_grad_(False)

In [ ]:
cbs = [partial(AvgStatsCallback,accuracy_flat),
       CudaCallback,
       Recorder,
       partial(GradientClipping, clip=0.1),
       partial(RNNTrainer, alpha=2., beta=1.),
       ProgressCallback]

In [ ]:
learn = Learner(model, data, cross_entropy_flat, opt_func=adam_opt(), cb_funcs=cbs, splitter=lm_splitter)

In [ ]:
lr = 2e-2
sched_lr  = combine_scheds([0.5,0.5], cos_1cycle_anneal(lr/10., lr, lr/1e5))
sched_mom = combine_scheds([0.5,0.5], cos_1cycle_anneal(0.8, 0.7, 0.8))
cbsched = [ParamScheduler('lr', sched_lr), ParamScheduler('mom', sched_mom)]

In [ ]:
learn.fit(1, cbs=cbsched)

epoch,train_loss,train_accuracy_flat,valid_loss,valid_accuracy_flat,time
0,4.485840,0.247460,4.280858,0.263456,07:19


In [ ]:
for rnn in model[0].rnns:
    for p in rnn.parameters(): p.requires_grad_(True)

In [ ]:
lr = 2e-3
sched_lr  = combine_scheds([0.25,0.75], cos_1cycle_anneal(lr/10.,lr, 0))
sched_lr1 = combine_scheds([0.25,0.75], cos_1cycle_anneal(lr/20.,lr/2., 0))
sched_mom = combine_scheds([0.25,0.75], cos_1cycle_anneal(0.8,0.7, 0.8))
cbsched = [ParamScheduler('lr', [sched_lr1, sched_lr1, sched_lr]), ParamScheduler('mom', sched_mom)]

In [ ]:
learn.fit(6, cbs=cbsched)

epoch,train_loss,train_accuracy_flat,valid_loss,valid_accuracy_flat,time
0,4.255009,0.263845,4.200441,0.271944,07:37
1,4.175897,0.272362,4.150997,0.277345,07:38
2,4.126333,0.277442,4.123255,0.280439,07:39
3,4.095077,0.280423,4.105435,0.282326,07:39
4,4.077273,0.282139,4.096895,0.283325,07:39
5,4.067260,0.283120,4.095329,0.283404,07:39


In [ ]:
learn.fit(10, cbs=cbsched)

epoch,train_loss,train_accuracy_flat,valid_loss,valid_accuracy_flat,time
0,4.262707,0.263002,4.215867,0.270290,07:32
1,4.196475,0.270250,4.167709,0.275524,07:34
2,4.145404,0.275602,4.132597,0.279453,07:35
3,4.107184,0.279326,4.110631,0.281655,07:35
4,4.079869,0.281920,4.094590,0.283156,07:36
5,4.059483,0.283736,4.081264,0.284467,07:36
6,4.044505,0.285149,4.072489,0.285487,07:36
7,4.031978,0.286326,4.067296,0.285958,07:36
8,4.025530,0.286830,4.064473,0.286331,07:36
9,4.022093,0.287139,4.064314,0.286363,07:37


In [ ]:
torch.save(learn.model[0].state_dict(), path/'finetuned_enc.pth')

In [ ]:
pickle.dump(vocab, open(path/'vocab_lm.pkl', 'wb'))

In [ ]:
torch.save(learn.model.state_dict(), path/'finetuned.pth')

## Classifier

We have to process the data again otherwise pickle will complain. We also have to use the same vocab as the language model.

In [ ]:
#vocab = pickle.load(open(path/'vocab_lm.pkl', 'rb'))

In [ ]:
il = TextList.from_files(path, include=['train', 'test'])
sd = SplitData.split_by_func(il, partial(grandparent_splitter, valid_name='test'))
ll = label_by_func(sd, parent_labeler, proc_x = [TokenizeProcessor(), NumericalizeProcessor()], #vocab=vocab 
                   proc_y=CategoryProcessor())

In [ ]:
pickle.dump(ll, open(path/'ll_clas.pkl', 'wb'))

In [ ]:
ll = pickle.load(open(path/'ll_clas.pkl', 'rb'))

In [ ]:
bs,bptt = 64,70
data = clas_databunchify(ll, bs)

### Ignore padding

In [ ]:
#export
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

In [ ]:
x,y = next(iter(data.train_dl))

In [ ]:
x.size()

torch.Size([64, 3311])

In [ ]:
lengths = x.size(1) - (x == 1).sum(1)
lengths[:5]

tensor([3311, 2425, 1782, 1521, 1513])

In [ ]:
tst_emb = nn.Embedding(len(vocab), 300)

In [ ]:
packed = pack_padded_sequence(tst_emb(x), lengths, batch_first=True)

In [ ]:
dp = nn.Dropout(p=0.5)

In [ ]:
tst = nn.LSTM(300, 300, 2)

In [ ]:
y,h = tst(packed)

In [ ]:
unpack = pad_packed_sequence(y, batch_first=True)

We need to change our model a little bit to use this.

In [ ]:
#export
class AWD_LSTM1(nn.Module):
    "AWD-LSTM inspired by https://arxiv.org/abs/1708.02182."
    initrange=0.1

    def __init__(self, vocab_sz, emb_sz, n_hid, n_layers, pad_token,
                 hidden_p=0.2, input_p=0.6, embed_p=0.1, weight_p=0.5):
        super().__init__()
        self.bs,self.emb_sz,self.n_hid,self.n_layers,self.pad_token = 1,emb_sz,n_hid,n_layers,pad_token
        self.encoder = nn.Embedding(vocab_sz, emb_sz, padding_idx=pad_token)
        self.encoder_dp = EmbeddingDropout(self.encoder, embed_p)
        self.rnns = [nn.LSTM(emb_sz if l == 0 else n_hid, (n_hid if l != n_layers - 1 else emb_sz), 1,
                             batch_first=True) for l in range(n_layers)]
        self.rnns = nn.ModuleList([WeightDropout(rnn, weight_p) for rnn in self.rnns])
        self.encoder.weight.data.uniform_(-self.initrange, self.initrange)
        self.input_dp = RNNDropout(input_p)
        self.hidden_dps = nn.ModuleList([RNNDropout(hidden_p) for l in range(n_layers)])

    def forward(self, input):
        self.bs = input.size(0)
        self.reset()
        mask = (input == self.pad_token)
        lengths = input.size(1) - mask.long().sum(1)
        raw_output = self.input_dp(self.encoder_dp(input))
        raw_outputs,outputs = [],[]
        for l, (rnn,hid_dp) in enumerate(zip(self.rnns, self.hidden_dps)):
            raw_output = pack_padded_sequence(raw_output, lengths, batch_first=True)
            raw_output, _ = rnn(raw_output, self.hidden[l])
            raw_output = pad_packed_sequence(raw_output, batch_first=True)[0]
            raw_outputs.append(raw_output)
            if l != self.n_layers - 1: raw_output = hid_dp(raw_output)
            outputs.append(raw_output)
        return raw_outputs, outputs, mask

    def _one_hidden(self, l):
        "Return one hidden state."
        nh = self.n_hid if l != self.n_layers - 1 else self.emb_sz
        return next(self.parameters()).new(1, self.bs, nh).zero_()

    def reset(self):
        "Reset the hidden states."
        self.hidden = [(self._one_hidden(l), self._one_hidden(l)) for l in range(self.n_layers)]

### Concat pooling

We will use three things for the classification head of the model: the last hidden state, the average of all the hidden states and the maximum of all the hidden states. The trick is just to, once again, ignore the padding in the average/maximum.

In [ ]:
def bn_drop_lin(n_in, n_out, bn=True, p=0., actn=None):
    layers = [nn.BatchNorm1d(n_in)] if bn else []
    if p != 0: layers.append(nn.Dropout(p))
    layers.append(nn.Linear(n_in, n_out))
    if actn is not None: layers.append(actn)
    return layers

In [ ]:
class PoolingLinearClassifier(nn.Module):
    "Create a linear classifier with pooling."

    def __init__(self, layers, drops):
        super().__init__()
        mod_layers = []
        activs = [nn.ReLU(inplace=True)] * (len(layers) - 2) + [None]
        for n_in, n_out, p, actn in zip(layers[:-1], layers[1:], drops, activs):
            mod_layers += bn_drop_lin(n_in, n_out, p=p, actn=actn)
        self.layers = nn.Sequential(*mod_layers)

    def forward(self, input):
        raw_outputs,outputs,mask = input
        output = outputs[-1]
        lengths = output.size(1) - mask.long().sum(dim=1)
        avg_pool = output.masked_fill(mask[:,:,None], 0).mean(dim=1)
        avg_pool *= output.size(1) / lengths.type(avg_pool.dtype)[:,None]
        max_pool = output.masked_fill(mask[:,:,None], -float('inf')).max(dim=1)[0]
        x = torch.cat([output[torch.arange(0, output.size(0)),lengths-1], max_pool, avg_pool], 1) #Concat pooling.
        x = self.layers(x)
        return x, raw_outputs, outputs

In [ ]:
x,y = next(iter(data.train_dl))

In [ ]:
mask = x==1

In [ ]:
mask.size()

torch.Size([16, 3311])

In [ ]:
lengths = x.size(1) - mask.long().sum(dim=1)

In [ ]:
z = torch.randn(3311)

In [ ]:
z.masked_fill(mask[1], 0).mean() * 3311/lengths[1]

tensor(0.0326)

In [ ]:
z[-lengths[1]:].mean()

tensor(0.0326)

In [ ]:
avg_pool = output.masked_fill(mask[:,:,None], 0).mean(dim=1)

Then we just have to feed our texts to those two blocks, but we can't give them all at once to the AWD_LSTM or we'll get OOM error: we'll go for chunks of bptt length to regularly detach the history of our hidden states.

In [ ]:
class SentenceEncoder(nn.Module):
    def __init__(self, bptt, module, max_batch=25):
        super().__init__()
        self.bptt,self.module,self.max_batch = bptt,module,max_batch
    
    def reset(self): 
        if hasattr(self.module, 'reset'): self.module.reset()
    
    def concat(self, arrs):
        return [torch.cat([l[i] for l in arrs], dim=1) for i in range(len(arrs[0]))]
    
    def forward(self, input):
        bs,sl = input.size()
        self.reset()
        raw_outputs,outputs,masks = [],[],[]
        for i in range(0, sl, self.bptt):
            r,o,m = self.module(input[:,i: min(i+self.bptt, sl)])
            #if i>(sl-self.bptt*self.max_len): #We only keep what is at the bptt * max_len last positions.
            masks.append(m)
            raw_outputs.append(r)
            outputs.append(o)
        return self.concat(raw_outputs),self.concat(outputs),torch.cat(masks,dim=1)

In [ ]:
def get_text_classifier(vocab_sz, emb_sz, n_hid, n_layers, n_out, pad_token, bptt, max_batch=25, layers=None,
                        drops=None, output_p=0.4, hidden_p=0.2, input_p=0.6, embed_p=0.1, weight_p=0.5):
    "To create a full AWD-LSTM"
    rnn_enc = AWD_LSTM1(vocab_sz, emb_sz, n_hid=n_hid, n_layers=n_layers, pad_token=pad_token,
                        hidden_p=hidden_p, input_p=input_p, embed_p=embed_p, weight_p=weight_p)
    #enc = SentenceEncoder(bptt, rnn_enc, max_batch=max_batch)
    if layers is None: layers = [50]
    if drops is None:  drops = [0.1] * len(layers)
    layers = [3 * emb_sz] + layers + [n_out] 
    drops = [output_p] + drops
    return SequentialRNN(rnn_enc, PoolingLinearClassifier(layers, drops))

In [ ]:
vocab = data.train_ds.proc_x[1].vocab

In [ ]:
emb_sz, nh, nl = 300, 300, 2
dps = tensor([0.4, 0.4, 0.5, 0.05, 0.3]) * 0.2
model = get_text_classifier(len(vocab), emb_sz, nh, nl, 2, 1, bptt, input_p=dps[0], output_p=dps[1], weight_p=dps[2], 
                           embed_p=dps[3], hidden_p=dps[4])

### Training

We load our pretrained encoder and freeze it.

In [ ]:
def class_splitter(m):
    groups = [nn.Sequential(m[0].encoder, m[0], m[0].input_dp)]
    for i in range(len(m[0].rnns)): groups.append(nn.Sequential(m[0].rnns[i], m[0].hidden_dps[i]))
    groups.append(m[1])
    return [list(o.parameters()) for o in groups]

In [ ]:
model[0].load_state_dict(torch.load(path/'finetuned_enc.pth'))

In [ ]:
for p in model[0].parameters(): p.requires_grad_(False)

In [ ]:
cbs = [partial(AvgStatsCallback,accuracy),
       CudaCallback,
       Recorder,
       partial(GradientClipping, clip=0.1),
       partial(RNNTrainer, alpha=0., beta=0.),
       ProgressCallback]

In [ ]:
learn = Learner(model, data, F.cross_entropy, lr=1e-2, opt_func=adam_opt(), cb_funcs=cbs, splitter=class_splitter)

In [ ]:
lr = 1e-3
sched_lr  = combine_scheds([0.5,0.5], cos_1cycle_anneal(lr/10., lr, lr/1e5))
sched_mom = combine_scheds([0.5,0.5], cos_1cycle_anneal(0.8, 0.7, 0.8))
cbsched = [ParamScheduler('lr', sched_lr), ParamScheduler('mom', sched_mom)]

In [ ]:
learn.fit(2, cbs=cbsched)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,0.698616,0.515064,0.745741,0.500280,01:06
1,0.684033,0.560657,0.687815,0.552520,01:06


In [ ]:
for p in model[0].rnns[-1].parameters(): p.requires_grad_(True)

In [ ]:
lr = 5e-3
sched_lr  = combine_scheds([0.25,0.75], cos_1cycle_anneal(lr/10., lr, lr/1e5))
sched_lr1  = combine_scheds([0.25,0.75], cos_1cycle_anneal(lr/20., lr/2, lr/2e5))
sched_mom = combine_scheds([0.25,0.75], cos_1cycle_anneal(0.8, 0.7, 0.8))
cbsched = [ParamScheduler('lr', [sched_lr1, sched_lr1, sched_lr1, sched_lr]), ParamScheduler('mom', sched_mom)]

In [ ]:
learn.fit(1, cbs=cbsched)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,0.294821,0.877160,1.523625,0.582520,00:57


In [ ]:
for p in model[0].rnns[-1].parameters(): p.requires_grad_(True)

In [ ]:
lr = 5e-3
sched_lr  = combine_scheds([0.25,0.75], cos_1cycle_anneal(lr/10., lr, lr/1e5))
sched_lr1  = combine_scheds([0.25,0.75], cos_1cycle_anneal(lr/20., lr/2, lr/2e5))
sched_mom = combine_scheds([0.25,0.75], cos_1cycle_anneal(0.8, 0.7, 0.8))
cbsched = [ParamScheduler('lr', [sched_lr1, sched_lr1, sched_lr1, sched_lr]), ParamScheduler('mom', sched_mom)]

In [ ]:
learn.fit(1, cbs=cbsched)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,0.338730,0.859000,0.211006,0.918760,02:23


In [ ]:
for p in model[0].parameters(): p.requires_grad_(True)

In [ ]:
lr = 1e-3
sched_lrs = [combine_scheds([0.25,0.75], cos_1cycle_anneal(lr/((2**i)*10.), lr/(2**i), lr/((2**i)*1e5))) for i in range(4)]
sched_lrs.reverse()
sched_mom = combine_scheds([0.25,0.75], cos_1cycle_anneal(0.8, 0.7, 0.8))
cbsched = [ParamScheduler('lr', sched_lrs), ParamScheduler('mom', sched_mom)]

In [ ]:
learn.fit(2, cbs=cbsched)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,0.292223,0.882160,0.228467,0.911200,03:33
1,0.269084,0.891840,0.191580,0.925920,03:33
